In [ ]:
# Author: Yu-Shin, Hu
# Date: March 21, 2022


# Remember to connect to NTU VPN

import requests
import pandas as pd
import datetime
import numpy as np
from datetime import datetime, timedelta

regions_dict = {
    'N1': '總配電站',
    'N2': '海洋所饋線',
    'N3': '推廣中心饋線',
    'N4': '管院教學館饋線',
    'N5': '生機系饋線',
    'N6': '霖澤饋線',
    'N7': '水源校區饋線',
    'N8': '南區',
    'N9': '台電獨立電號',
    'NA': '宿舍區',
    'NB': '徐州校區',
    'all': '全校館舍'
}

buildings_dict = {
    'N1': ['數學館', '新數學研究中心', '-新生大樓-', '生化所', '電機一館', '化學館', '貴儀中心氦液化機用電', '化學館(不含貴儀中心氦液化機用電)', '積學館', '博雅館', '思亮館', '-原分所-', '物理學系/凝態中心停車場', '凝態物理科學館', '心理南研究大樓', '女八九舍(含女九餐廳)', '心理系北館', '應用力學研究大樓', '工學院綜合大樓'],
    'N2': ['全球變遷中心', '海洋研究所', '游泳池', '舊體育館', '-闈場-', '普通大樓', '小福樓', '化學工程館', '土木館', '文學院', '司令台', '農業陳列館', '校史館', '人類學博物館', '舊總圖', '樂學館', '新南球場', '新月台', '新南停車場', '紅土網球場', '新月台揚水及汙水泵用電'],    'N2': ['全球變遷中心', '海洋研究所', '游泳池', '舊體育館', '-闈場-', '普通大樓', '小福樓', '化學工程館', '土木館', '文學院', '司令台', '農業陳列館', '校史館', '人類學博物館', '舊總圖', '樂學館', '新南球場', '新月台', '新南停車場', '紅土網球場', '新月台揚水及汙水泵用電'],
    'N3': ['女五舍', '三號館', '-電子顯微鏡館-', '農化二館暨行政大樓', '性別平等教育委員會', '物理系液氮機室', '-推廣中心-', '-城鄉所-', '大門口警衛室', '一號館', '一號館1F(戲劇系)', '一號館2F', '一號館3F', '動物標本館', '漁業陳列館', '-漁業生物實驗室-', '女一舍', '女三舍', '女二舍', '植物病毒研究室', '植物溫室', '植物標本館', '椰林大道路燈前段', '雜工班', '二號館', '2號館1F', '2號館2F', '2號館3F'],
    'N4': ['農化系食品工廠', '化工光電晶體實驗室', '生醫材料(陶瓷)實驗室', '綠色化學實驗室', '職涯中心', '學生心理輔導中心', '望樂樓', '行政大樓教務處資訊室', '農化系實驗室', '夜間部大樓', '保管組倉庫', '小小福', '舟山路崗亭', '行政大樓前棟', '行政大樓後棟', '第一會議室', '四號館', '五號館', '農綜館', '植物工廠', '共同教室', '農產品中心', '生工系實驗室', '農化系土壤研究室', '森林館', '水工試驗所', '水工試驗所馬達房', '保健中心', '林產館', '航空測量館', '停車場路燈', '管理學院教學館', '原教堂(藝文中心)', '多功能生活廳', '水工大樓', '園藝系花卉館'],
    'N5': ['總圖書館', '資訊工程館', '-機械臨時工廠-', '電機二館', '聯合研究中心', '獸醫一館', '獸醫三館', '志鴻館', '機械舊館', '振興草坪路燈', '水杉道路燈', '綜合教室', '圖書資訊館', '第一活動中心', '農藝館', '教學二期大樓(不含圖書館書庫)', '圖書館書庫', '鄭江樓北棟化工系(不含大公)', '鄭江樓南棟生機系(不含公共)', '鄭江樓南棟6樓保管組(不含公共)', '鄭江樓南棟7樓保管組(不含公共)', '鄭江樓南棟空調', '鄭江樓南棟公共(揚水、電梯)', '鄭江樓事務組', '鄭江樓大公(雨廢水、消防)', '學新館', '農機館', '中非大樓', '博理館'],
    'N6': ['社會研究所停車場', '社會研究所', '-辛亥大門-', '國發所停車場', '國家發展所大樓', '新聞所停車場', '新聞所大樓', '-水產養殖池-', '漁業科學館', '計資中心', '視聽教育館', '語言大樓', '霖澤館停車場', '霖澤館', '社科院大樓總錶', '萬才館停車場', '萬才館'],
    'N7': ['水源校區總圖(飲水樓)', '水源溫室', '機械工廠', '澄思樓', '輔具中心', '水源事務組停車場', '育成大樓(水源)', '行政大樓(水源)', '理化大樓(水源)', '育成C', '卓越研究大樓'],
    'N8': ['學生第二活動中心', '管院二號館', '展書樓', '戲劇廳（鹿嗚堂二樓）', '鹿鳴堂一樓', '鹿鳴雅舍', '教職員工聯誼廳', '地質系', '幼稚園', '園藝系造園館', '食科館', '園產加工廠', '食科所食品研發大樓', '大氣系A、B館', '園藝系精密溫室', '轉殖溫室', '精密溫室', '環工所', '工科海洋系', '禮賢樓1樓總用電(不含空調)', '禮賢樓8樓總用電', '禮賢樓劇場用電', '禮賢樓大公共用電', '禮賢樓1樓空調用電', '禮賢樓1樓銀行專用迴路', '禮賢樓2樓總用電', '禮賢樓3樓總用電', '禮賢樓4樓總用電', '禮賢樓5樓總用電', '禮賢樓6樓總用電', '禮賢樓7樓總用電', '動物醫院', '動科系', '生技中心', '昆蟲館', '禮賢樓總用電'],
    'N9': ['土木研究大樓', '環安衛中心', '環工所', '動物實驗中心', '國青舍（研三舍）', '體育館', '管理學院1號館', '地理系', '浩翰樓華南銀行', '華南銀行司機室', '浩翰樓', '立體機車停車場總用電', '立體機車停車場事務組用電(不含大公)', '立體機車停車場工科海洋用電(不含大公)', '立體機車停車場大公共用電', '獸醫二館', '明達館', '明達館停車場', '明達2F保管組場地', '玉山台大AI暨金融科技研發中心', '生科館停車場', '生科館', '農業試驗場', '種子研究室', '人工氣候室'],
    'NA': ['研一女', '研一男', '男一舍', '男一舍外大餐廳', '男三舍', '男五舍', '男七舍', '男八舍', '男六舍', '大一女舍'],
    'NB': ['經研大樓', '徐州校區國際會議廳', '法學院社科圖書館'],
    # 'all': ['數學館', '新數學研究中心', '-新生大樓-', '生化所', '電機一館', '化學館', '貴儀中心氦液化機用電', '化學館(不含貴儀中心氦液化機用電)', '積學館', '博雅館', '思亮館', '-原分所-', '物理學系/凝態中心停車場', '凝態物理科學館', '心理南研究大樓', '女八九舍(含女九餐廳)', '心理系北館', '應用力學研究大樓', '工學院綜合大樓', '研一女', '研一男', '男一舍', '男一舍外大餐廳', '男三舍', '男五舍', '男七舍', '男八舍', '男六舍', '大一女舍', '全球變遷中心', '海洋研究所', '游泳池', '舊體育館', '-闈場-', '普通大樓', '小福樓', '化學工程館', '土木館', '文學院', '司令台', '農業陳列館', '校史館', '人類學博物館', '舊總圖', '樂學館', '新南球場', '新月台', '新南停車場', '紅土網球場', '新月台揚水及汙水泵用電', '女五舍', '三號館', '-電子顯微鏡館-', '農化二館暨行政大樓', '性別平等教育委員會', '物理系液氮機室', '-推廣中心-', '-城鄉所-', '大門口警衛室', '一號館', '一號館1F(戲劇系)', '一號館2F', '一號館3F', '動物標本館', '漁業陳列館', '-漁業生物實驗室-', '女一舍', '女三舍', '女二舍', '植物病毒研究室', '植物溫室', '植物標本館', '椰林大道路燈前段', '雜工班', '二號館', '2號館1F', '2號館2F', '2號館3F', '農化系食品工廠', '化工光電晶體實驗室', '生醫材料(陶瓷)實驗室', '綠色化學實驗室', '職涯中心  ', '學生心理輔導中心', '望樂樓', '行政大樓教務處資訊室', '農化系實驗室', '夜間部大樓', '保管組倉庫', '小小福', '舟山路崗亭', '行政大樓前棟', '行政大樓後棟', '第一會議室', '四號館', '五號館', '農綜館', '植物工廠', '共同教室', '農產品中心', '生工系實驗室', '農化系土壤研究室', '森林館', '水工試驗所', '水工試驗所馬達房', '保健中心', '林產館', '航空測量館', '停車場路燈', '管理學院教學館', '原教堂(藝文中心)', '多功能生活廳', '水工大樓', '園藝系花卉館', '總圖書館', '資訊工程館', '-機械臨時工廠-', '電機二館', '聯合研究中心', '獸醫一館', '獸醫三館', '志鴻館', '機械舊館', '振興草坪路燈', '水杉道路燈', '綜合教室', '圖書資訊館', '第一活動中心', '農藝館', '教學二期大樓(不含圖書館書庫)', '圖書館書庫', '鄭江樓北棟化工系(不含大公)', '鄭江樓南棟生機系(不含公共)', '鄭江樓南棟6樓保管組(不含公共)', '鄭江樓南棟7樓保管組(不含公共)', '鄭江樓南棟空調', '鄭江樓南棟公共(揚水、電梯)', '鄭江樓事務組', '鄭江樓大公(雨廢水、消防)', '學新館', '農機館', '中非大樓', '博理館', '社會研究所停車場', '社會研究所', '-辛亥大門-', '國發所停車場', '國家發展所大樓', '新聞所停車場', '新聞所大樓', '-水產養殖池-', '漁業科學館', '計資中心', '視聽教育館', '語言大樓', '霖澤館停車場', '霖澤館', '社科院大樓總錶', '萬才館停車場', '萬才館', '水源校區總圖(飲水樓)', '水源溫室', '機械工廠', '澄思樓', '輔具中心', '水源事務組停車場', '育成大樓(水源)', '行政大樓(水源)', '理化大樓(水源)', '育成C', '卓越研究大樓', '學生第二活動中心', '管院二號館', '展書樓', '戲劇廳（鹿嗚堂二樓）', '鹿鳴堂一樓', '-鹿鳴雅舍-', '教職員工聯誼廳', '地質系', '幼稚園', '園藝系造園館', '食科館', '園產加工廠', '食科所食品研發大樓', '大氣系A、B館', '園藝系精密溫室', '轉殖溫室', '精密溫室', '環工所', '工科海洋系', '禮賢樓1樓總用電(不含空調)', '禮賢樓8樓總用電', '禮賢樓劇場用電', '禮賢樓大公共用電', '禮賢樓1樓空調用電', '禮賢樓1樓銀行專用迴路', '禮賢樓2樓總用電', '禮賢樓3樓總用電', '禮賢樓4樓總用電', '禮賢樓5樓總用電', '禮賢樓6樓總用電', '禮賢樓7樓總用電', '動物醫院', '動科系', '生技中心', '昆蟲館', '土木研究大樓', '環安衛中心', '環工所', '動物實驗中心', '國青舍（研三舍）', '體育館', '管理學院1號館', '地理系', '浩翰樓華南銀行', '華南銀行司機室', '浩翰樓', '立體機車停車場總用電', '立體機車停車場事務組用電(不含大公)', '立體機車停車場工科海洋用電(不含大公)', '立體機車停車場大公共用電', '獸醫二館', '明達館', '明達館停車場', '明達2F保管組場地', '玉山台大AI暨金融科技研發中心', '生科館停車場', '生科館', '禮賢樓總用電', '經研大樓', '徐州校區國際會議廳', '法學院社科圖書館', '農業試驗場', '種子研究室', '人工氣候室'],
}

# Enter parameters
start_date = "2017/01/01"
end_date = "2017/01/02"
b = '數學館'
path = r'C:/Users/Username/Downloads/test/' # Choose path in the computer

# Date
start_date = datetime.strptime(start_date, "%Y/%m/%d")
end_date = datetime.strptime(end_date, "%Y/%m/%d")
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
date_range = [date.strftime("%Y/%m/%d") for date in date_range]

# Region of the building
r = [key for key, value in buildings_dict.items() if b in value][0]

data_output = []
for d in date_range:
    
    # Read the website
    url = 'https://epower.ga.ntu.edu.tw/fn4/report2.aspx' # Our URL
    payload = {
        'ctg': r,
        'dt1': d,
        'ok': '確定',
    }
    read_url = requests.post(url, data=payload)
    data = pd.read_html(read_url.text)[1]

    # Select a building
    data_temp = data.copy()
    data_temp.columns = data_temp.iloc[1]
    data_temp = data_temp.iloc[3:27]
    data_temp = data_temp[b]
    data_temp = [float(val) if val != '---' else np.nan for val in data_temp] # data_temp = list(map(float, data_temp))
    data_output += data_temp

# Output datafrme
output = pd.DataFrame()
output['Datetime'] = pd.date_range(start=start_date, end=end_date + pd.Timedelta(hours=23), freq='H')
output[b] = pd.DataFrame(data_output)
output.to_csv( path + b + ".csv", index = False)

print( "DONE" )
